<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/BERT_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd
import random 
import re
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
## 1) Sentences will be tokenized using the TextVectorization 
## 2) 12.5 % of the tokens will be replaced with the [MASK] token. Substituded tokens will be 
## chosen randomly. To select the random tokens, random sampling will be chosen 

## 3) segment id will be determined at the second step
## 4) positional embeddings will be directly taken from the article

In [3]:
trial_sentences = [
    'my name is john',
    'london is the capital is the england',
    'weather is cold at the northern hemisphere of the world',
    'today, i will go the school'
]

longest_sentence = []
for i in trial_sentences:
    longest_sentence.append(len(i.split()))
    
print(np.max(longest_sentence))

longest_sentence = np.max(longest_sentence)

10


In [4]:
vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
                            max_tokens = 30,
#                             standardize=LOWER_AND_STRIP_PUNCTUATION,
#                             split=SPLIT_ON_WHITESPACE
                            output_sequence_length = int(longest_sentence)
)

vectorizer.adapt(trial_sentences)

In [5]:
tf_tokenizer = Tokenizer()
tf_tokens=tf_tokenizer.fit_on_texts(trial_sentences)

corpus = tf_tokenizer.word_index
extended_corpus = corpus.copy()

extended_corpus['MASK'] = len(corpus)+1

corpus_tokens = []
for i,j in enumerate(corpus):
    corpus_tokens.append(j)

In [6]:
extended_tokens = []
for i,j in  enumerate(extended_corpus):
    print(i,j)
    extended_tokens.append(j)
    
subs_tokens=random.sample(corpus_tokens,int(len(corpus_tokens)*0.12))
print(subs_tokens)

sequences = tf_tokenizer.texts_to_sequences(trial_sentences)
padded_sequences= pad_sequences(sequences)
padded_sequences

substitute_padded_sequences= padded_sequences.copy()

substitute_token_ids = [ ]

for i in subs_tokens:
    substitute_token_ids.append(tf_tokenizer.word_index[i])
    
    
mask_token_id = 20
print(substitute_token_ids)

0 the
1 is
2 my
3 name
4 john
5 london
6 capital
7 england
8 weather
9 cold
10 at
11 northern
12 hemisphere
13 of
14 world
15 today
16 i
17 will
18 go
19 school
20 MASK
['name', 'the']
[4, 1]


In [8]:
for i in range(len(extended_tokens)):
    if i == (substitute_token_ids[0]-1 or substitute_token_ids[1]-1  ):
        print(i)

3
